# 📊 Exemplo de Coleta de Dados - Big Data Finance

Este notebook demonstra como usar os módulos de coleta de dados do projeto Big Data Finance.

## Objetivos
- Coletar dados financeiros do Yahoo Finance
- Coletar notícias e eventos mundiais
- Salvar dados em diferentes formatos
- Demonstrar rate limiting e tratamento de erros

**Autor:** Felipe Martins (APIs e Coleta de Dados) & Equipe Big Data Finance  
**Gestão:** Fabio  
**Infraestrutura:** Ana Luiza Pazze  
**Data:** 2024

In [ ]:
# Imports necessários
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Adicionar src ao path
sys.path.append('../src')

# Imports dos módulos do projeto
from data_collection.yahoo_finance_collector import YahooFinanceCollector
from data_collection.news_collector import NewsCollector

print("✅ Imports realizados com sucesso!")

## 1. 📈 Coleta de Dados Financeiros

Vamos começar coletando dados financeiros usando o Yahoo Finance API.

In [ ]:
# Inicializar o coletor Yahoo Finance
yahoo_collector = YahooFinanceCollector(rate_limit=1.0)

# Definir período de análise
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

print(f"📅 Período de análise: {start_date} até {end_date}")
print(f"🔧 Rate limit configurado: {yahoo_collector.rate_limit} segundos")

### 1.1 Coleta de Dados de Ações

In [ ]:
# Símbolos de ações para análise
stock_symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']

print(f"📊 Coletando dados para: {', '.join(stock_symbols)}")
print("⏳ Aguarde... (pode levar alguns minutos devido ao rate limiting)")

# Coletar dados de ações
stock_data = yahoo_collector.collect_stock_data(
    symbols=stock_symbols,
    start_date=start_date,
    end_date=end_date
)

print(f"✅ Dados coletados: {len(stock_data)} registros")
print(f"📋 Colunas disponíveis: {list(stock_data.columns)}")

# Visualizar primeiros registros
display(stock_data.head())

### 1.2 Coleta de Dados de Índices

In [ ]:
# Índices principais
index_symbols = ['^GSPC', '^DJI', '^IXIC', '^RUT']
index_names = ['S&P 500', 'Dow Jones', 'NASDAQ', 'Russell 2000']

print(f"📈 Coletando dados de índices: {', '.join(index_names)}")

# Coletar dados de índices
index_data = yahoo_collector.collect_index_data(
    symbols=index_symbols,
    start_date=start_date,
    end_date=end_date
)

print(f"✅ Dados de índices coletados: {len(index_data)} registros")

# Visualizar dados
display(index_data.head())

### 1.3 Informações das Empresas

In [ ]:
# Coletar informações detalhadas das empresas
company_info = {}

for symbol in stock_symbols[:3]:  # Apenas 3 para exemplo
    print(f"🏢 Coletando informações de {symbol}...")
    info = yahoo_collector.collect_company_info(symbol)
    company_info[symbol] = info

# Criar DataFrame com informações principais
info_df = pd.DataFrame({
    symbol: {
        'Nome': info.get('longName', 'N/A'),
        'Setor': info.get('sector', 'N/A'),
        'Indústria': info.get('industry', 'N/A'),
        'Market Cap': info.get('marketCap', 'N/A'),
        'P/E Ratio': info.get('trailingPE', 'N/A')
    } for symbol, info in company_info.items()
}).T

print("📋 Informações das empresas:")
display(info_df)

## 2. 📰 Coleta de Notícias e Eventos

Agora vamos coletar notícias e eventos mundiais que podem impactar os mercados.

In [ ]:
# Inicializar coletor de notícias
news_collector = NewsCollector()

# Palavras-chave para eventos relevantes
keywords = [
    'Federal Reserve', 'inflation', 'GDP', 'unemployment',
    'trade war', 'COVID', 'recession', 'interest rates'
]

print(f"📰 Coletando eventos com palavras-chave: {', '.join(keywords)}")
print("⏳ Aguarde... (pode levar alguns minutos)")

In [ ]:
# Coletar eventos históricos
events_data = news_collector.collect_historical_events(
    start_date=start_date,
    end_date=end_date,
    keywords=keywords
)

print(f"✅ Eventos coletados: {len(events_data)} registros")
print(f"📋 Colunas disponíveis: {list(events_data.columns)}")

# Visualizar eventos
if not events_data.empty:
    display(events_data.head())
else:
    print("⚠️ Nenhum evento encontrado (pode ser devido a limitações da API)")

## 3. 📊 Análise Exploratória dos Dados Coletados

Vamos fazer uma análise inicial dos dados coletados.

### 3.1 Análise dos Dados Financeiros

In [ ]:
# Estatísticas descritivas
print("📊 Estatísticas Descritivas dos Preços de Fechamento:")
print("=" * 60)

# Filtrar apenas preços de fechamento
close_prices = stock_data.pivot_table(
    index='date', 
    columns='symbol', 
    values='close'
)

display(close_prices.describe())

In [ ]:
# Gráfico de preços normalizados
plt.figure(figsize=(12, 8))

# Normalizar preços para base 100
normalized_prices = close_prices.div(close_prices.iloc[0]) * 100

for symbol in normalized_prices.columns:
    plt.plot(normalized_prices.index, normalized_prices[symbol], 
             label=symbol, linewidth=2)

plt.title('📈 Performance Normalizada das Ações (Base 100)', fontsize=16, fontweight='bold')
plt.xlabel('Data', fontsize=12)
plt.ylabel('Preço Normalizado', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Performance no período
performance = ((close_prices.iloc[-1] / close_prices.iloc[0]) - 1) * 100
print("\n🎯 Performance no Período (%)")
print("=" * 30)
for symbol, perf in performance.items():
    print(f"{symbol}: {perf:+.2f}%")

### 3.2 Análise de Volume

In [ ]:
# Análise de volume
volume_data = stock_data.pivot_table(
    index='date', 
    columns='symbol', 
    values='volume'
)

plt.figure(figsize=(12, 6))

# Volume médio por ação
avg_volume = volume_data.mean()
avg_volume.plot(kind='bar', color='skyblue', alpha=0.7)

plt.title('📊 Volume Médio de Negociação por Ação', fontsize=16, fontweight='bold')
plt.xlabel('Símbolo', fontsize=12)
plt.ylabel('Volume Médio', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Adicionar valores nas barras
for i, v in enumerate(avg_volume):
    plt.text(i, v + v*0.01, f'{v:,.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

### 3.3 Análise de Volatilidade

In [ ]:
# Calcular retornos diários
returns = close_prices.pct_change().dropna()

# Volatilidade (desvio padrão anualizado)
volatility = returns.std() * np.sqrt(252) * 100

plt.figure(figsize=(10, 6))
volatility.plot(kind='bar', color='coral', alpha=0.7)

plt.title('📈 Volatilidade Anualizada por Ação (%)', fontsize=16, fontweight='bold')
plt.xlabel('Símbolo', fontsize=12)
plt.ylabel('Volatilidade (%)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Adicionar valores nas barras
for i, v in enumerate(volatility):
    plt.text(i, v + v*0.01, f'{v:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n📊 Ranking de Volatilidade:")
print("=" * 30)
for i, (symbol, vol) in enumerate(volatility.sort_values(ascending=False).items(), 1):
    print(f"{i}. {symbol}: {vol:.2f}%")

### 3.4 Matriz de Correlação

In [ ]:
# Matriz de correlação dos retornos
correlation_matrix = returns.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True, 
            linewidths=0.5,
            fmt='.3f')

plt.title('🔗 Matriz de Correlação dos Retornos', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Pares mais correlacionados
print("\n🔗 Pares Mais Correlacionados:")
print("=" * 35)

# Extrair correlações (excluindo diagonal)
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        symbol1 = correlation_matrix.columns[i]
        symbol2 = correlation_matrix.columns[j]
        corr_value = correlation_matrix.iloc[i, j]
        corr_pairs.append((symbol1, symbol2, corr_value))

# Ordenar por correlação
corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

for symbol1, symbol2, corr in corr_pairs[:5]:
    print(f"{symbol1} - {symbol2}: {corr:.3f}")

## 4. 💾 Salvamento dos Dados

Vamos salvar os dados coletados em diferentes formatos para uso posterior.

In [ ]:
# Criar diretório de dados se não existir
data_dir = '../data/raw'
os.makedirs(data_dir, exist_ok=True)

# Salvar dados de ações
stock_file = f"{data_dir}/stock_data_{datetime.now().strftime('%Y%m%d')}.csv"
stock_data.to_csv(stock_file, index=False)
print(f"✅ Dados de ações salvos: {stock_file}")

# Salvar dados de índices
index_file = f"{data_dir}/index_data_{datetime.now().strftime('%Y%m%d')}.csv"
index_data.to_csv(index_file, index=False)
print(f"✅ Dados de índices salvos: {index_file}")

# Salvar informações das empresas
info_file = f"{data_dir}/company_info_{datetime.now().strftime('%Y%m%d')}.csv"
info_df.to_csv(info_file)
print(f"✅ Informações das empresas salvas: {info_file}")

# Salvar eventos (se houver)
if not events_data.empty:
    events_file = f"{data_dir}/events_data_{datetime.now().strftime('%Y%m%d')}.csv"
    events_data.to_csv(events_file, index=False)
    print(f"✅ Dados de eventos salvos: {events_file}")

print(f"\n📁 Todos os dados salvos em: {os.path.abspath(data_dir)}")

## 5. 📋 Resumo e Próximos Passos

### ✅ O que foi realizado:
1. **Coleta de dados financeiros** usando Yahoo Finance API
2. **Coleta de informações de empresas** (setor, indústria, métricas)
3. **Coleta de dados de índices** principais do mercado
4. **Tentativa de coleta de eventos** (limitada por APIs)
5. **Análise exploratória** dos dados coletados
6. **Salvamento dos dados** em formato CSV

### 📊 Estatísticas da coleta:

In [ ]:
# Resumo estatístico
print("📊 RESUMO DA COLETA DE DADOS")
print("=" * 50)
print(f"📈 Ações coletadas: {len(stock_symbols)} símbolos")
print(f"📊 Registros de ações: {len(stock_data):,}")
print(f"📈 Índices coletados: {len(index_symbols)} símbolos")
print(f"📊 Registros de índices: {len(index_data):,}")
print(f"🏢 Empresas analisadas: {len(company_info)}")
print(f"📰 Eventos coletados: {len(events_data) if not events_data.empty else 0}")
print(f"📅 Período analisado: {start_date} a {end_date}")
print(f"📁 Arquivos salvos: {4 if not events_data.empty else 3}")

print("\n🎯 PRÓXIMOS PASSOS:")
print("=" * 30)
print("1. 🔄 Processar dados com Apache Spark")
print("2. 📊 Análise estatística avançada")
print("3. 🤖 Aplicar modelos de Machine Learning")
print("4. 💭 Análise de sentimentos das notícias")
print("5. 📈 Criar visualizações interativas")
print("6. 🔗 Analisar correlações com eventos")

print("\n✨ Notebook concluído com sucesso!")

---

## 📚 Referências e Links Úteis

- **Yahoo Finance API**: [yfinance documentation](https://pypi.org/project/yfinance/)
- **Pandas**: [pandas.pydata.org](https://pandas.pydata.org/)
- **Matplotlib**: [matplotlib.org](https://matplotlib.org/)
- **Seaborn**: [seaborn.pydata.org](https://seaborn.pydata.org/)

---

**Desenvolvido pela Equipe Big Data Finance**  
**Notebook:** 01_data_collection_example.ipynb  
**Versão:** 1.0